TODO: just load a regular linear layer, a classic MLP, and a GAT model and run some graphs through it 

TODO: explain what needs to happen in training, validation and test steps with a diagram

In [ ]:
import torch
import pytorch_lightning as L
from torch_geometric.nn import GAT
import torch.nn.functional as F

class GATModule(L.LightningModule):
    """
    LightningModule wrapping a GAT model.
    """
    def __init__(self, in_channels, hidden_channels, num_layers, num_heads, out_channels, dropout, jk):
        super(GATModule, self).__init__()
        self.save_hyperparameters()
        self.example_input_array = (torch.empty(32, in_channels), torch.zeros(2, 100, dtype=torch.long))
        self.model = GAT(in_channels=in_channels,
                         hidden_channels=hidden_channels,
                         num_layers=num_layers,
                         heads=num_heads,
                         out_channels=out_channels,
                         dropout=dropout,
                         jk=jk, v2=True) # TODO: try other models

    def forward(self, node_attributes, edge_index):
        return self.model(node_attributes, edge_index)

    def training_step(self, batch, batch_idx):
        out = self(batch.x, batch.edge_index)
        loss = F.binary_cross_entropy_with_logits(out, batch.y.view(-1, 1))
        self.log('train_loss', loss, on_step=True, on_epoch=True, sync_dist=True,
                 batch_size=batch.x.shape[0])

        return loss

    def validation_step(self, batch, batch_idx):
        out = self(batch.x, batch.edge_index)
        loss = F.binary_cross_entropy_with_logits(out, batch.y.view(-1, 1))
        self.log('hp_metric', loss, on_step=True, on_epoch=True, sync_dist=True,
                 batch_size=batch.x.shape[0])
        return loss

    def configure_optimizers(self):
      return torch.optim.Adam(params=self.parameters(), lr=0.001, weight_decay=0.0001)

TODO: explain losses

In [ ]:
from src.dataloader import ProteinDataModule

def train_model():
  model = GATModule(
    in_channels=20,
    num_layers=3,
    hidden_channels=128,
    num_heads=4,
    out_channels=1,
    dropout=0.01,
    jk='cat',
  )

  dataloader = ProteinDataModule(
    root="./data",
    dataset_file="dataset.txt",
    batch_size=32,
    num_workers=4,
  )

  trainer = L.Trainer(
      devices="auto",
      accelerator="auto",
      enable_progress_bar=True,
      max_epochs=5,
  )

  trainer.fit(model=model, datamodule=dataloader)

# TODO: Other models, add custom layers, losses etc.

## Bonus:
- how would you change things for protein-protein input?